In [2]:
# !pip install gdown

In [ ]:
import gdown
url = "https://drive.google.com/u/0/uc?id=1Oqt3Kpl8W0OR715ac_evMv8GQHvnXWO5&export=download"
output = "cropped_video.zip"
gdown.download(url, output)

In [ ]:
%%capture
!pip install pytorch-lightning -q
!pip install neptune-client -q
!pip install iopath -q
!pip !install av -q
!git clone https://github.com/talhaanwarch/pytorchvideo.git
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113 --upgrade
!pip install torchtext==0.12.0 --upgrade
!pip install fvcore -q
!pip install wandb -q
!pip install av -q
# !pip install gdown
!pip install matplotlib
!pip install -U albumentations
!pip install openpyxl pandas
# !pip install pytorchvideo
# !pip uninstall pytorchvideo --y

In [ ]:
import numpy as np
from pytorch_lightning import seed_everything, LightningModule, Trainer
import torch.nn as nn
import torch
from torch.utils.data.dataloader import DataLoader
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,LearningRateMonitor
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau,CosineAnnealingWarmRestarts,OneCycleLR,CosineAnnealingLR
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
from PIL import Image
import cv2
import os
from torch.utils.data import DataLoader, Dataset,ConcatDataset
import albumentations as A
from sklearn.model_selection import KFold,GroupShuffleSplit,GroupKFold,LeaveOneGroupOut
from torchmetrics import MeanAbsoluteError
from sklearn.utils import shuffle
import shutil
from sklearn.model_selection import train_test_split



In [ ]:
!mkdir ./temp
!sudo apt-get install unzip -y
!unzip -d ./temp/ cropped.zip

In [ ]:
import pandas as pd
df=pd.read_csv('new_file.csv')

In [ ]:
df.info()

In [ ]:
vital='Pulse'
df=df[['path',vital]]
df.columns=['path','value']
df.path='temp/'+df['path']

In [ ]:
indexpath = df[ (df['value'] < 60)].index
# & (df['value'] < 89) ].index
df.drop(indexpath , inplace=True)
# df.head(15)

In [ ]:
indexpath=df[(df["value"]>125)].index
df.drop(indexpath, inplace=True)

In [ ]:
df.hist()

In [ ]:
count_nan = df.isnull().sum()
print ('Count of NaN: ' + str(count_nan))

In [ ]:
df.dropna(inplace=True)

In [ ]:
train_dt,val_dt=train_test_split(df,test_size=0.33,random_state=0)

In [ ]:
import sys
sys.path.insert(0, "pytorchvideo/")
print(sys.path)

In [ ]:

from pytorchvideo.data import LabeledVideoDataset,Kinetics, make_clip_sampler
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,

)

In [ ]:
num_video_samples=16
video_duration=5
model_name='pul_x3d'
batch_size=16
# batch_size=32
scheduler='cosine'
clipmode='uniform'
img_size=256
augmentation=False

In [ ]:
import wandb
wandb.login()

In [ ]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler,labeled_video_dataset
from torchvision.transforms import ColorJitter,RandomAdjustSharpness,RandomAutocontrast,GaussianBlur
train_transform = Compose(
            [
            ApplyTransformToKey(
              key="video",
              transform=Compose(
                  [
                    UniformTemporalSubsample(num_video_samples),
                    Lambda(lambda x: x / 255.0),
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                    RandomShortSideScale(min_size=img_size+16, max_size=img_size+32),
                    CenterCropVideo(img_size),
                    Permute([1,0,2,3]),
                    RandomAdjustSharpness(1.2),
                    RandomAutocontrast(),
                    Permute([1,0,2,3]),
                    RandomHorizontalFlip(p=0.5)
                    # GaussianBlur(kernel_size=(7, 13), sigma=(0.1, 0.2))

                  ]
                ),
              ),
            ]
        )



In [ ]:
#tuneable params
# num_video_samples=24
from time import sleep
import torchvision.models as models
# from scripts.cnnlstm import CNNLSTM
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel,self).__init__()

        self.scheduler=scheduler
        self.metric=MeanAbsoluteError()
        self.model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)
        self.model.blocks[5].proj=nn.Linear(in_features=2048, out_features=1, bias=True)

        # self.model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=False)
        # self.model.blocks[5]=nn.Sequential(*list(self.model.blocks[5].children())[:-2])
        # self.model.blocks[5][2]=nn.Sequential(nn.AdaptiveAvgPool3d(1),
        #                          nn.Flatten(),
        #                          nn.Linear(2048,1024),
        #                          nn.ReLU(),
        #                          nn.Linear(1024,1))

        self.lr=1e-3
        self.batch_size=batch_size
        self.numworker=2
        self.criterion=nn.MSELoss(reduction='mean')
    def forward(self,x):
        return self.model(x)

    def configure_optimizers(self):
        opt=torch.optim.AdamW(params=self.parameters(),lr=self.lr )
        if self.scheduler=='cosine':
            scheduler=CosineAnnealingLR(opt,T_max=10,  eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='reduce':
            scheduler=ReduceLROnPlateau(opt,mode='min', factor=0.5, patience=5)
            return {'optimizer': opt,'lr_scheduler':scheduler,'monitor':'val_loss'}
        elif self.scheduler=='warm':
            scheduler=CosineAnnealingWarmRestarts(opt,T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='cycle':
            opt=torch.optim.AdamW(params=self.parameters(),lr=1e-6 )
            scheduler=OneCycleLR(opt,max_lr=1e-2,epochs=15,steps_per_epoch=len(self.train_df)//self.batch_size//4)
            lr_scheduler = {'scheduler': scheduler, 'interval': 'step'}
            return {'optimizer': opt, 'lr_scheduler': lr_scheduler}
        elif self.scheduler=='lambda':
            lambda1 = lambda epoch: 0.9 ** epoch
            scheduler = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda=lambda1)
            return {'optimizer': opt, 'lr_scheduler': scheduler}
        elif self.scheduler=='constant':
            return opt

    def train_dataloader(self):
        train_dataset=labeled_video_dataset(train_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=train_transform, decode_audio=False)

        train_loader=DataLoader(train_dataset,batch_size=self.batch_size,
                   num_workers=self.numworker,
                   pin_memory=False)
        return train_loader

    def training_step(self,batch,batch_idx):
        out = self(batch["video"])
#         print("out")
        label=batch['label'].float()
#         print("label")
        # label=batch['label'].int()
        loss=self.criterion(out,label)
#         print("loss")
        mae=self.metric(out,label)
#         print("mae")
        # mae=self.metric(torch.exp(out),torch.exp(label))
        return {'loss':loss,'mae':mae.detach()}

    def training_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().cpu().numpy().round(2)
        mae=torch.stack([x["mae"] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('train_loss', loss,batch_size=self.batch_size)
        self.log('train_mae', mae,batch_size=self.batch_size)
        print('training loss ',self.current_epoch,loss,mae)

    def val_dataloader(self):
        val_dataset=labeled_video_dataset(val_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=train_transform,  decode_audio=False)

        val_loader=DataLoader(val_dataset,shuffle=False,
                   batch_size=self.batch_size,
                   num_workers=self.numworker,
                   pin_memory=False)
        return val_loader

    def validation_step(self,batch,batch_idx):
        out = self(batch["video"])
#         print("validation")
        # label=batch['label'].int()
        label=batch['label'].float()
#         print("label")
        loss=self.criterion(out,label)
#         print("lossss")
        # mae=self.metric(torch.exp(out),torch.exp(label))
        mae=self.metric(out,label)
#         print("maaa")
        return {'loss':loss,'mae':mae.detach()}

    def validation_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().cpu().numpy().round(2)
        mae=torch.stack([x["mae"] for x in outputs]).mean().cpu().numpy().round(2)
        print('validation loss ',self.current_epoch,loss,mae)
        self.log('val_loss', loss,batch_size=self.batch_size)
        self.log('val_mae',mae,batch_size=self.batch_size)


In [ ]:
model=OurModel()
seed_everything(0)
lr_monitor = LearningRateMonitor(logging_interval='epoch')
trainer = Trainer(max_epochs=60,
#         deterministic=True,
        gpus=1,precision=16,
        accumulate_grad_batches=2,
        enable_progress_bar = True,
        num_sanity_val_steps=0,
        #gradient_clip_val=0.5,
        callbacks=[lr_monitor]
#         logger=wandb_logger
                                    )
trainer.fit(model)
res=trainer.validate(model)

In [ ]:
onnx='temp/'+'_'.join([model_name,scheduler,clipmode,'pulse_v3.onnx'])
input_sample = torch.randn((1,3,16,256,256))
model.to_onnx(onnx, input_sample,opset_version=11, export_params=True)

In [ ]:
 wandb_logger.experiment.config.update({
"num_video_samples": num_video_samples,
"video_duration":video_duration,
"batch_size":batch_size,
"clipmode":clipmode,
"scheduler": scheduler,
"augmentation":augmentation,
"val_mae": res[0]['val_mae'],
"val_loss": res[0]['val_loss']})

In [ ]:
wandb_logger.experiment.save(pt)

In [ ]:
test_dataset=labeled_video_dataset(val_df,
                   clip_sampler=make_clip_sampler('random', video_duration),\
                    transform=train_transform,  decode_audio=False)

val_loader=DataLoader(test_dataset,batch_size=1,
           num_workers=2,
           pin_memory=True)

In [ ]:
preds,labels,abss=[],[],[]
model.eval()
model=model.cuda()
with torch.no_grad():
    for batch in val_loader:
        pred=np.exp(model(batch['video']).cpu().detach().numpy()).astype(int)
        label=int(np.exp(batch['label'].numpy()))
    #     print('true:',label,'pred',pred)

        abss.append(np.abs(label-pred))
        preds.append(pred)
        labels.append(label)

In [ ]:
print('abs',np.max(abss),np.median(abss),np.min(abss),np.mean(abss).round(1))
print('label',np.max(labels),np.min(labels),np.mean(labels).round(1))
print('preds',np.max(preds),np.min(preds),np.mean(preds).round(1))
print('preds_diff',np.max(preds)-np.min(preds))

In [ ]:
wandb_logger.experiment.config.update({
"max_abs": np.max(abss),
"mean_abs":np.mean(abss),
"median_abs":np.median(abss),
"max_pred":np.max(preds),
"min_pred":np.min(preds),
"dif_pred":np.max(preds)-np.min(preds)
})


In [ ]:
out_df=pd.DataFrame(zip(labels,preds,abss),columns=['labels','preds','diff'])
out_df['labels']=out_df['labels'].apply(lambda x:x[0])
out_df['preds']=out_df['preds'].apply(lambda x:x[0])
out_df['diff']=out_df['diff'].apply(lambda x:x[0])

In [ ]:
wandb_logger.log_table(key="samples", columns=list(out_df.columns), data=out_df)
out_df.head()